In [1]:
import urllib2
import json
import datetime
import time
import threading
import timeit
import pandas as pd
import cPickle as pk
from time import strftime,strptime,gmtime
import datetime

import numpy as np
import scipy as sp

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# Visualization libraries
# Displays plots in output cell
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing, cross_validation
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, SGDClassifier
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor

In [2]:
def positive_negative (v):
    if v >= 0:
        return 1
    else: 
        return 0 

In [3]:
def learning_downloader (stock_list,days = 10000,shift_days = 10):
    #date variables
    dt = datetime.datetime.now()
    UnixTime = int(time.mktime(dt.timetuple()))
    data_df = pd.DataFrame()
    for s in stock_list:
        #web variables
        url = 'https://query1.finance.yahoo.com/v8/finance/chart/'+s+'?period1='+str(UnixTime-86400*days)+'&period2='+str(UnixTime)+'&interval=1d&indicators=quote%7Csma~50&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-CA&region=CA&corsDomain=ca.finance.yahoo.com'
        hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36',
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
               'Connection': 'keep-alive'}
        #html data request 
        try:
            request = urllib2.Request(url,headers = hdr)
            htmltext = urllib2.urlopen(request)
        
            data = json.load(htmltext)
            #json unpack
            timestamp = data['chart']['result'][0]['timestamp']
            timestamp = [datetime.datetime.fromtimestamp(x).strftime('%Y%m%d') for x in timestamp]
            quote = data['chart']['result'][0]['indicators']['quote'][0]
            sma50 = data['chart']['result'][0]['indicators']['sma'][0]['sma']
            stock_df = pd.DataFrame(quote)
            #index is symbol and timestamp
            stock_df.index = [s + str(x) for x in timestamp]

            #moving averages
            stock_df['sma'] = sma50
            stock_df['sma'] = stock_df['sma'].shift(1)
            stock_df['vol20'] = stock_df['volume'].rolling(window=20).mean()
            stock_df = stock_df.dropna()

            #derived columns
            stock_df['c_0'] = stock_df['close']/stock_df['sma']
            stock_df['h_0'] = stock_df['high']/stock_df['sma']
            stock_df['l_0'] = stock_df['low']/stock_df['sma']
            stock_df['o_0'] = stock_df['open']/stock_df['sma']
            stock_df['v_0'] = stock_df['volume']/stock_df['vol20']
            stock_df['r'] = (stock_df['close']-stock_df['open'])/stock_df['open']

            #rolling columns
            for d in list(range(1,shift_days+1)):
                stock_df['c_'+str(d)] = stock_df['c_0'].shift(d)
                stock_df['h_'+str(d)] = stock_df['h_0'].shift(d)
                stock_df['l_'+str(d)] = stock_df['l_0'].shift(d)
                stock_df['o_'+str(d)] = stock_df['o_0'].shift(d)
                stock_df['v_'+str(d)] = stock_df['v_0'].shift(d)
            stock_df.drop(['close', 'high', 'low', 'open','volume','sma','vol20'], axis=1, inplace=True)
            stock_df = stock_df.dropna()

            #add to main df 
            if len(data_df) == 0:
                data_df = stock_df
            elif timestamp is None : 
                pass
            else:
                data_df = data_df.append(stock_df)
            
        except:
            pass
            
    data_df = data_df.dropna()
    return data_df
        

In [17]:
def stock_predict (symbol,clf,clfR,shift_days=10,days_ago = 0):
    #date variables
    dt = datetime.datetime.now()
    UnixTime = int(time.mktime(dt.timetuple()))
    #web variables
    url = 'https://query1.finance.yahoo.com/v8/finance/chart/'+symbol+'?period1='+str(UnixTime-86400*(60+days_ago))+'&period2='+str(UnixTime-86400*days_ago)+'&interval=1d&indicators=quote%7Csma~50&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-CA&region=CA&corsDomain=ca.finance.yahoo.com'
    hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
           'Connection': 'keep-alive'}
    #html data request 
    try:
        request = urllib2.Request(url,headers = hdr)
        htmltext = urllib2.urlopen(request)

        data = json.load(htmltext)
        #json unpack
        timestamp = data['chart']['result'][0]['timestamp']
        timestamp = [datetime.datetime.fromtimestamp(x).strftime('%Y%m%d') for x in timestamp]
        quote = data['chart']['result'][0]['indicators']['quote'][0]
        sma50 = data['chart']['result'][0]['indicators']['sma'][0]['sma']
        stock_df = pd.DataFrame(quote)
        #index is symbol and timestamp
        stock_df.index = [symbol + str(x) for x in timestamp]

        #moving averages
        stock_df['sma'] = sma50
        stock_df['sma'] = stock_df['sma'].shift(1)
        stock_df['vol20'] = stock_df['volume'].rolling(window=20).mean()
        stock_df = stock_df.dropna()

        #derived columns
        stock_df['c_0'] = stock_df['close']/stock_df['sma']
        stock_df['h_0'] = stock_df['high']/stock_df['sma']
        stock_df['l_0'] = stock_df['low']/stock_df['sma']
        stock_df['o_0'] = stock_df['open']/stock_df['sma']
        stock_df['v_0'] = stock_df['volume']/stock_df['vol20']

        #rolling columns
        for d in list(range(1,shift_days+1)):
            stock_df['c_'+str(d)] = stock_df['c_0'].shift(d)
            stock_df['h_'+str(d)] = stock_df['h_0'].shift(d)
            stock_df['l_'+str(d)] = stock_df['l_0'].shift(d)
            stock_df['o_'+str(d)] = stock_df['o_0'].shift(d)
            stock_df['v_'+str(d)] = stock_df['v_0'].shift(d)
        stock_df.drop(['close', 'high', 'low', 'open','volume','sma','vol20','c_0','v_0','h_0','l_0'], axis=1, inplace=True)
        stock_df = stock_df.dropna()
    except:
        pass
    #select last row of downloaded matrix for latest day open price and rolling data
    return '%s will go %s by %s, prediction date: %s'%(symbol, str(clf.predict(stock_df.iloc[[-1]].values.tolist())),str(clfR.predict(stock_df.iloc[[-1]].values.tolist())),timestamp[-1])

In [80]:
#download learning data from list of stocks
#stock_list = ['AA', 'ABT', 'ABX', 'ADI', 'ADM', 'AET', 'AMD', 'AMR', 'APC', 'AVP', 'AXP', 'BA', 'BAC', 'BAX', 'BBY', 'BK', 'BMC', 'BMY', 'BNI', 'BP', 'CA', 'CAT', 'CI', 'CL', 'COP', 'CVX', 'DD', 'DE', 'DIS', 'DOW', 'EK', 'EMC', 'EMR', 'FNM', 'FRE', 'FRX', 'GE', 'GLW', 'GPS', 'GSK', 'HAL', 'HD', 'HON', 'HPQ', 'HRB', 'IBM', 'IGT', 'JNJ', 'JPM', 'JWN', 'KO', 'KR', 'LLY', 'LOW', 'LTD', 'LUV', 'MCD', 'MDT', 'MMM', 'MO', 'MOT', 'MRK', 'MRO', 'MU', 'MYL', 'NKE', 'NSM', 'NWS', 'OXY', 'PEP', 'PFE', 'PG', 'RSH', 'SLB', 'SLE', 'SLM', 'STJ', 'SYK', 'SYY', 'TGT', 'TJX', 'TMX', 'TXN', 'UN', 'UNH', 'UTX', 'VOD', 'VZ', 'WAG', 'WFC', 'WMB', 'WMT', 'XOM', 'XRX']
#stock_list = ['^GDAXI','^FTSE','^DJI','^FCHI','^N225','^HSI','^AXJO','^GSPC','^IXIC','^TNX','^VIX']
stock_list = ['ifmk']
ml_df = learning_downloader(stock_list)
#not making decision on intraday close high low and volumes 
ml_df = ml_df.drop(['c_0','v_0','h_0','l_0'], axis=1)

#regression learning data prep
Xr = np.array(ml_df.drop(['r'],1))
yr = np.array(ml_df['r'])
Xr_train, Xr_test, yr_train, yr_test = cross_validation.train_test_split(Xr,yr,test_size= 0.2)

#classification data prep
ml_df['r'] = ml_df['r'].map(positive_negative)
X = np.array(ml_df.drop(['r'],1))
y = np.array(ml_df['r'])
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size= 0.2)

#Gradiant Boosting Ensemble learning
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train,y_train)
print 'classification score: %s'%(clf.score(X_test, y_test))
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X,y)

#GBE regression learning
clfR = GradientBoostingRegressor(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(Xr_train,yr_train)
print 'regression score: %s'%(clfR.score(Xr_test, yr_test))
clfR = GradientBoostingRegressor(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(Xr,yr)

classification score: 0.530612244898
regression score: -0.578284646646


In [81]:
#prediction testing
dt = datetime.datetime.now()
UnixTime = int(time.mktime(dt.timetuple()))
symbol = 'ifmk'

for d in list(range(1,30))[::-1]:
    print stock_predict(symbol,clf,clfR,days_ago=d)

#web variables
url = 'https://query1.finance.yahoo.com/v8/finance/chart/'+symbol+'?period1='+str(UnixTime-86400*30)+'&period2='+str(UnixTime)+'&interval=1d&indicators=quote%7Csma~50&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-CA&region=CA&corsDomain=ca.finance.yahoo.com'
hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
'Connection': 'keep-alive'}
#html data request 
request = urllib2.Request(url,headers = hdr)
htmltext = urllib2.urlopen(request)

data = json.load(htmltext)
#json unpack
timestamp = data['chart']['result'][0]['timestamp']
timestamp = [datetime.datetime.fromtimestamp(x).strftime('%Y%m%d') for x in timestamp]
quote = data['chart']['result'][0]['indicators']['quote'][0]
stock_df = pd.DataFrame(quote)
#index is symbol and timestamp
stock_df.index = [symbol + str(x) for x in timestamp]
stock_df = stock_df.dropna()
stock_df['r'] = (stock_df['close']-stock_df['open'])/stock_df['open']
print stock_df

ifmk will go [0] by [-0.01857054], prediction date: 20180223
ifmk will go [0] by [-0.01857054], prediction date: 20180223
ifmk will go [1] by [-0.00191985], prediction date: 20180226
ifmk will go [0] by [-0.02004376], prediction date: 20180227
ifmk will go [0] by [-0.02837127], prediction date: 20180228
ifmk will go [0] by [-0.00323922], prediction date: 20180301
ifmk will go [0] by [-0.01272641], prediction date: 20180302
ifmk will go [0] by [-0.01272641], prediction date: 20180302
ifmk will go [0] by [-0.01272641], prediction date: 20180302
ifmk will go [0] by [-0.0587876], prediction date: 20180305
ifmk will go [0] by [-0.04500926], prediction date: 20180306
ifmk will go [0] by [-0.0390539], prediction date: 20180307
ifmk will go [0] by [-0.03698017], prediction date: 20180308
ifmk will go [1] by [ 0.06732344], prediction date: 20180309
ifmk will go [1] by [ 0.06732344], prediction date: 20180309
ifmk will go [1] by [ 0.06732344], prediction date: 20180309
ifmk will go [1] by [ 0.01